In [2]:
import pandas as pd
dfTrades = pd.read_csv("/Users/rohitk/Documents/Sure Data/IBMandRHTTrades.csv")
dfQuotes = pd.read_csv("/Users/rohitk/Documents/Sure Data/IBMandRHTQuotes.csv")

In [3]:
dfTrades = dfTrades.drop(columns = ['SYM_SUFFIX'])
dfTrades['Datetime'] = pd.to_datetime(dfTrades["DATE"].astype(str) + " " + dfTrades["TIME_M"].astype(str))
dfTrades = dfTrades.drop(columns=['DATE','TIME_M'])
dfTrades = dfTrades.set_index('Datetime')

In [4]:
dfQuotes = dfQuotes.drop(columns = ['SYM_SUFFIX'])
dfQuotes['Datetime'] = pd.to_datetime(dfQuotes["DATE"].astype(str) + " " + dfQuotes["TIME_M"].astype(str))
dfQuotes = dfQuotes.drop(columns=['DATE','TIME_M'])
dfQuotes = dfQuotes.set_index('Datetime')

In [5]:
dfRHTTrades = dfTrades[dfTrades["SYM_ROOT"] == "RHT"]
dfIBMTrades = dfTrades[dfTrades["SYM_ROOT"] == "IBM"]

dfRHTQuotes = dfQuotes[dfQuotes["SYM_ROOT"] == "RHT"]
dfIBMQuotes = dfQuotes[dfQuotes["SYM_ROOT"] == "IBM"]

In [27]:
a = pd.merge_asof(dfRHTTrades,dfRHTQuotes,left_index=True,right_index=True,direction='nearest')
a= a.drop(columns=['SYM_ROOT_y'])
a.rename(columns={'SYM_ROOT_x':'Symbol'}, inplace=True)
a["Direction"] = ""

,Symbol,SIZE,PRICE,BID,BIDSIZ,ASK,ASKSIZ,Direction
Datetime,,,,,,,,
2018-10-22 08:59:06.735286,RHT,3,120.20,120.12,2,130.27,1,
2018-10-22 09:12:27.156497,RHT,1,120.12,120.13,1,122.49,1,
2018-10-22 09:23:32.457412,RHT,3,120.12,120.12,1,130.27,1,
2018-10-22 09:30:00.809023,RHT,14686,120.77,120.56,1,121.37,2,
2018-10-22 09:30:01.261716,RHT,20,120.56,0.00,0,121.29,2,
...,...,...,...,...,...,...,...,...
2018-11-02 18:05:15.571047,RHT,60599,172.24,169.91,2,173.00,1,
2018-11-02 18:17:24.482159,RHT,11,172.20,169.91,2,173.00,1,
2018-11-02 18:17:24.482241,RHT,11,172.18,169.91,2,173.00,1,


In [57]:
#Lee-Ready classification algorithm

#If price above bid ask midpoint -> buy
# If price below bid ask midpoint -> sell
# If price equals midpoint see if trade price is higher or lower than previous
# If equal to previous keep going till you find higher or lower trade price
# higher price -> buy
# lower price -> sell
b = a.copy()
b = b[:1000]

prevPrice = 0
for index, row in b.iterrows():
    midpoint = (row['BID'] + row['ASK']) / 2

    if row['PRICE'] > midpoint:
        b.loc[index,'Direction'] = 'B'
    elif row['PRICE'] < midpoint:
        b.loc[index,'Direction'] = 'S'
    else:
        if prevPrice == 0:
            b.loc[index,'Direction'] = ''
        else:
            if row['PRICE'] > prevPrice:
                b.loc[index,'Direction'] = 'B'
            elif row['PRICE'] < prevPrice:
                b.loc[index,'Direction'] = 'S'
            else:
                b.loc[index,'Direction'] = ''
    prevPrice = row['PRICE']

print(len(b[b['Direction'] == '']))

6
